## Vision por Computador: Práctica 6

# Juego razas

### Variables e importaciones necesarias

In [29]:
# Importa los modulos necesarios
import cv2
import os
from deepface import DeepFace

# Cargamos los gorros guardados
gorros = {
        'middle eastern': cv2.imread('assets/gorro_ruso.png', cv2.IMREAD_UNCHANGED),
        'asian': cv2.imread('assets/gorro_chino.png', cv2.IMREAD_UNCHANGED),
        'indian': cv2.imread('assets/gorro_indio.png', cv2.IMREAD_UNCHANGED),
        'latino hispanic': cv2.imread('assets/gorro_mexicano.png', cv2.IMREAD_UNCHANGED),
        'black': cv2.imread('assets/gorro_negro.png', cv2.IMREAD_UNCHANGED),
        'white': cv2.imread('assets/gorro_blanco.png', cv2.IMREAD_UNCHANGED),
        'default': cv2.imread('assets/gorro_blanco.png', cv2.IMREAD_UNCHANGED)
        }

for nombre, gorro in gorros.items():
    print(f"{nombre}: {gorro.shape}")  # Debería mostrar algo como (altura, anchura, 4)



class Button:
    def __init__(self, x1, y1, size="normal"):
        if size == "normal":
            self.width = 245
            self.height = 105
        elif size == "small":
            self.width = 160
            self.height = 80
        elif size == "squared":
            self.width = 110
            self.height = 110
        else:
            raise ValueError("Tamaño de botón no válido")
        
        self.x1 = x1
        self.y1 = y1
        
    def get_coordinates(self):
        x2 = self.x1 + self.width
        y2 = self.y1 + self.height
        return (self.x1, self.y1, x2, y2)

    
botones_menu = {
    "ELEGIR": Button(x1=50, y1=285).get_coordinates(),  # (x1, y1, x2, y2) (35, 195, 210, 270)
    "DETECTAR": Button(x1=325, y1=285).get_coordinates(),
    "SALIR": Button(x1=600, y1=285).get_coordinates(),
}

botones_detectar = {
    "MENU": Button(x1=35, y1=580, size="small").get_coordinates(),
    "SALIR": Button(x1=710, y1=580, size="small").get_coordinates()
}

botones_elegir = {
    "MENU": Button(x1=35, y1=15, size="small").get_coordinates(),
    "SALIR": Button(x1=710, y1=15, size="small").get_coordinates()
}

botones_cambiar_sombrero = {
    'black': Button(x1=55, y1=545, size="squared").get_coordinates(),
    'indian': Button(x1=190, y1=545, size="squared").get_coordinates(),
    'asian':Button(x1=325, y1=545, size="squared").get_coordinates(),
    'latino hispanic': Button(x1=460, y1=545, size="squared").get_coordinates(),
    'middle eastern':Button(x1=595, y1=545, size="squared").get_coordinates(),
    'white': Button(x1=735, y1=545, size="squared").get_coordinates(),
}

# Cargar la imagen de los botones_menu 
pantalla_menu = cv2.imread('assets/pantalla_menu.png', cv2.IMREAD_UNCHANGED)

# Cargar la imagen de los botones_detectar
pantalla_detectar = cv2.imread('assets/pantalla_detectar.png', cv2.IMREAD_UNCHANGED)

# Cargar las imágenes de fondo para cada selección de gorro
pantallas_elegir = {
    'white': cv2.imread('assets/GorroBlancoSeleccionado.png', cv2.IMREAD_UNCHANGED),
    'indian': cv2.imread('assets/GorroIndioSeleccionado.png', cv2.IMREAD_UNCHANGED),
    'asian': cv2.imread('assets/GorroChinoSeleccionado.png', cv2.IMREAD_UNCHANGED),
    'latino hispanic': cv2.imread('assets/GorroMexicanoSeleccionado.png', cv2.IMREAD_UNCHANGED),
    'black': cv2.imread('assets/GorroNegroSeleccionado.png', cv2.IMREAD_UNCHANGED),
    'middle eastern': cv2.imread('assets/GorroRusoSeleccionado.png', cv2.IMREAD_UNCHANGED)
}

# Estado del juego
estado = "MENU"  # Estados posibles: "MENU", "DETECTAR", "ELEGIR", "SALIR"
# Diccionario para almacenar el gorro seleccionado
gorro_seleccionado = 'white'


middle eastern: (612, 612, 4)
asian: (512, 512, 4)
indian: (724, 600, 4)
latino hispanic: (1790, 1920, 4)
black: (1595, 1920, 4)
white: (375, 385, 4)
default: (375, 385, 4)


In [30]:
def superponer_imagen(fondo, superposicion):
    
    superposicion = cv2.resize(superposicion, (fondo.shape[1], fondo.shape[0]))
    # Extraer el canal alpha de la superposición
    alpha_s = superposicion[:, :, 3] / 255.0
    alpha_l = 1 - alpha_s

    # Superponer la imagen
    for c in range(0, 3):
        fondo[:, :, c] = (alpha_s * superposicion[:, :, c] +
                          alpha_l * fondo[:, :, c])
        
def superponer_gorro(frame, gorro, x, y, w, h):
    gorro_redimensionado = cv2.resize(gorro, (w, int(h / 2)))
    for i in range(gorro_redimensionado.shape[0]):
        for j in range(gorro_redimensionado.shape[1]):
            if gorro_redimensionado[i, j, 3] != 0:
                frame[y - int(h / 2) + i, x + j, :] = gorro_redimensionado[i, j, 0:3]
    return frame

def __detectar(frame):
    # Variable estática para el conteo de frames
    if 'contador_frames' not in detectar.__dict__:
        detectar.contador_frames = 0  # Inicializar en la primera llamada
    if 'obj' not in detectar.__dict__:
        detectar.obj = DeepFace.analyze(img_path=frame, enforce_detection=False, actions=['race'])  # Almacenar los últimos resultados de DeepFace
        
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    font, fontScale, color, thickness, pos = cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, (50, 50)


    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(100, 100))

    # Incrementar contador de frames
    detectar.contador_frames += 1
    
    # Aplicar DeepFace solo cada 50 frames
    if detectar.contador_frames % 40 == 0 and len(faces) > 0:
        # Analizar todas las caras a la vez
        detectar.obj = DeepFace.analyze(img_path=frame, enforce_detection=False, actions=['race'])
        print(detectar.obj)
        #print(obj)

    for i, (x, y, w, h) in enumerate(faces):
        if i < len(detectar.obj):
            # Obtenemos la raza dominante para la cara actual
            raza_dominante = detectar.obj[i]["dominant_race"]
            gorro = gorros.get(raza_dominante, gorros['default'])
            
            frame = superponer_gorro(frame, gorro, x, y, w, h)
            #cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            #cv2.putText(frame, raza_dominante, pos, font, fontScale, color, thickness, cv2.LINE_AA)
        else:
            break  # No hay más resultados de DeepFace para procesar
        
    return frame

def detectar(frame):
    if 'contador_frames' not in detectar.__dict__:
        detectar.contador_frames = 0  # Inicializar en la primera llamada
    if 'obj' not in detectar.__dict__:
        detectar.obj = DeepFace.analyze(img_path=frame, enforce_detection=False, actions=['race'])  # Almacenar los últimos resultados de DeepFace
    
    # Incrementar contador de frames
    detectar.contador_frames += 1
    
    # Aplicar DeepFace solo cada 40 frames
    if detectar.contador_frames % 5 == 0:
        # Analizar todas las caras a la vez
        detectar.obj = DeepFace.analyze(img_path=frame, enforce_detection=False, actions=['race'])
        print(detectar.obj)

    if detectar.obj:
        for resultado in detectar.obj:
            confianza = resultado['face_confidence']
            if confianza >= 2:
                # Extraer la información de la región y la raza dominante
                region = resultado['region']
                raza_dominante = resultado['dominant_race']
    
                # Coordenadas de la cara
                x, y, w, h = region['x'], region['y'], region['w'], region['h']
    
                # Seleccionar el gorro basado en la raza dominante
                gorro = gorros.get(raza_dominante, gorros['default'])
    
                # Superponer el gorro en la cara detectada
                frame = superponer_gorro(frame, gorro, x, y, w, h)
                # Opcional: Dibujar rectángulo y texto
                # cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                # font, fontScale, color, thickness, pos = cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, (50, 50)
                # cv2.putText(frame, raza_dominante, (x, y - 10), font, fontScale, color, thickness, cv2.LINE_AA)
        
    return frame




def elegir(frame):
    global gorro_seleccionado
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=6, minSize=(150, 150))

    # Superponer el gorro seleccionado sobre cada cara detectada
    for (x, y, w, h) in faces:
        if gorro_seleccionado is not None:
            gorro = gorros[gorro_seleccionado]
            frame = superponer_gorro(frame, gorro, x, y, w, h)

    return frame

def salir():
    print("Saliendo del juego")
    cv2.destroyAllWindows()


# Función para dibujar los botones_menu en el frame
def dibujar_botones(frame):
    global gorro_seleccionado
    if estado == "MENU":
        superponer_imagen(fondo=frame, superposicion=pantalla_menu)
        for nombre_boton, (x1, y1, x2, y2) in botones_menu.items():
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
    if estado == "DETECTAR":
        superponer_imagen(fondo=frame, superposicion=pantalla_detectar)
        for nombre_boton, (x1, y1, x2, y2) in botones_detectar.items():
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
    if estado == "ELEGIR":
        superponer_imagen(fondo=frame, superposicion=pantallas_elegir[gorro_seleccionado])        
        for nombre_boton, (x1, y1, x2, y2) in botones_elegir.items():
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2) 
        for nombre_boton, (x1, y1, x2, y2) in botones_cambiar_sombrero.items():
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)     
        
# Función de devolución de llamada para manejar eventos de clic del mouse
def clic_mouse(event, x, y, flags, param):
    global estado, gorro_seleccionado
    if event == cv2.EVENT_LBUTTONDOWN:
        if estado == "MENU":
            # Aquí detectas en qué botón se hizo clic y actualizas el estado
            for nombre_boton, (x1, y1, x2, y2) in botones_menu.items():
                if x1 < x < x2 and y1 < y < y2:
                    estado = nombre_boton  # Actualiza el estado basado en el botón presionado
        elif estado == "DETECTAR":
            for nombre_boton, (x1, y1, x2, y2) in botones_detectar.items():
                if x1 < x < x2 and y1 < y < y2:  
                    estado = nombre_boton  # Actualiza el estado basado en el botón presionado
        elif estado == "ELEGIR":
            for nombre_gorro, (x1, y1, x2, y2) in botones_cambiar_sombrero.items():
                if x1 < x < x2 and y1 < y < y2:
                    gorro_seleccionado = nombre_gorro
                    break
            for nombre_boton, (x1, y1, x2, y2) in botones_elegir.items():
                if x1 < x < x2 and y1 < y < y2:
                    estado = nombre_boton  # Actualiza el estado basado en el botón presionado
                    break
                   


In [31]:
# ==================================CÓDIGO PRINCIPAL==================================            

def iniciar_juego(fuente_video=0):
    global estado
    estado = "MENU"
    # Inicializar la fuente de video (cámara o archivo de video)
    vid = cv2.VideoCapture(fuente_video)

    if not vid.isOpened():
        print("No se pudo abrir la fuente de video:", fuente_video)
        return

     # Asumir que todas las pantallas superpuestas tienen la misma resolución
    altura_pantalla, ancho_pantalla = pantalla_menu.shape[:2]
    
     # Imprimir la resolución del frame actual
    print(f"Resolución de la pantalla: {ancho_pantalla}x{altura_pantalla}")

    # Establecer la función de callback para el mouse
    cv2.namedWindow('Juego')
    cv2.setMouseCallback('Juego', clic_mouse)

    while True:
        ret, frame = vid.read()
        
        if not ret:
            print("No se pudo leer el frame. Fin del video o error de lectura.")
            break
        
        # Redimensionamos el frame para que sea del tamaño de la pantalla de juego
        #frame = cv2.resize(frame, (ancho_pantalla,altura_pantalla))
        #print(f"Resolución del frame: {frame.shape[1]}x{frame.shape[0]}")
        
        # Manejar el estado del juego
        
        if estado == "MENU":
            dibujar_botones(frame)
        elif estado == "DETECTAR":
            detectar(frame) 
            dibujar_botones(frame)
        elif estado == "ELEGIR":
            elegir(frame)  
            dibujar_botones(frame)
        elif estado == "SALIR":
            break
        
        cv2.imshow('Juego', frame)

        if cv2.waitKey(1) & 0xFF == 27:
            break
        
                
    vid.release()
    cv2.destroyAllWindows()

# Ejemplo de cómo usar la función
# Para usar la cámara por defecto
#iniciar_juego()


# Para usar un archivo de vídeo
iniciar_juego('./video_test_5.mp4')

#iniciar_juego('./video_test_4.mp4')




Resolución de la pantalla: 900x699
[{'race': {'asian': 34.18595100557512, 'indian': 2.270200238835826, 'black': 6.934854801586557, 'white': 39.46852323225279, 'middle eastern': 7.637506433576729, 'latino hispanic': 9.502961307940561}, 'dominant_race': 'white', 'region': {'x': 0, 'y': 0, 'w': 1280, 'h': 640}, 'face_confidence': 0}]
[{'race': {'asian': 26.274041907432192, 'indian': 18.660373607896187, 'black': 38.44411304259974, 'white': 1.8306519064695779, 'middle eastern': 1.8772273512022108, 'latino hispanic': 12.913597399806822}, 'dominant_race': 'black', 'region': {'x': 458, 'y': 110, 'w': 246, 'h': 246}, 'face_confidence': 7.889253028144594}]
[{'race': {'asian': 19.383516087112895, 'indian': 14.384649469298187, 'black': 55.01344144184095, 'white': 1.1526138079030743, 'middle eastern': 1.2317969857994335, 'latino hispanic': 8.83397578192046}, 'dominant_race': 'black', 'region': {'x': 478, 'y': 101, 'w': 243, 'h': 243}, 'face_confidence': 9.253905119781848}]
[{'race': {'asian': 51.75